<a href="https://colab.research.google.com/github/thinhngo-x/Anonymization/blob/master/notebook/NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")
root = "/content/drive/My Drive/Anonymization"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import torch
from torch import nn

In [38]:
dataset = "/CONLL2003_BERT"

train = np.load(root+dataset+"/representation.train.npy")
train_labels = np.load(root+dataset+"/true_labels.train.npy")
# train_labels = np.logical_or(train_labels == 3, train_labels == 4)

test = np.load(root+dataset+"/representation.testb.npy")
test_labels = np.load(root+dataset+"/true_labels.testb.npy")
# test_labels = np.logical_or(test_labels == 3, test_labels == 4)

# random_projection = GaussianRandomProjection(n_components=10)
# train_projected = random_projection.fit_transform(train)
# test_projected = random_projection.transform(test)

print(train.shape)
# print(train_projected.shape)

print(test.shape)
# print(test_projected.shape)

(203593, 1024)
(46417, 1024)


In [0]:
train = torch.Tensor(train)
train_labels = torch.Tensor(train_labels).type(torch.LongTensor)
test = torch.Tensor(test)
test_labels = torch.Tensor(test_labels).type(torch.LongTensor)

In [0]:
fc = nn.Sequential(
    nn.Linear(1024, 128),
    nn.ReLU(),
    nn.Linear(128, 9),
    nn.LogSoftmax(1)
)
optim = torch.optim.Adam(fc.parameters())
criterion = nn.CrossEntropyLoss()
if torch.cuda.is_available():
    fc = fc.cuda()

In [41]:
epocs = 600
for i in range (epocs):
    optim.zero_grad()
    preds = fc(train.cuda())
    loss = criterion(preds, train_labels.cuda())
    if i % 50 == 0:
        print("[%d/%d]: Loss = %f" %(i, epocs, loss.item()))
    loss.backward()
    optim.step()

[0/500]: Loss = 2.236522
[50/500]: Loss = 0.219776
[100/500]: Loss = 0.140024
[150/500]: Loss = 0.111548
[200/500]: Loss = 0.095441
[250/500]: Loss = 0.084274
[300/500]: Loss = 0.075831
[350/500]: Loss = 0.068997
[400/500]: Loss = 0.063102
[450/500]: Loss = 0.058048
[500/500]: Loss = 0.053595
[550/500]: Loss = 0.049642


In [42]:
with torch.no_grad():
    preds = fc(test.cuda()).detach().cpu().numpy()
preds = np.argmax(preds, 1)
preds.shape

(46417,)

In [43]:
from sklearn.metrics import classification_report

print(classification_report(test_labels, preds,
            labels = range(9),
            target_names=["O", "B-MISC", "I-MISC", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC"]))

              precision    recall  f1-score   support

           O       0.99      0.99      0.99     38312
      B-MISC       0.00      0.00      0.00         9
      I-MISC       0.68      0.72      0.70       909
       B-PER       0.00      0.00      0.00         0
       I-PER       0.93      0.90      0.91      2769
       B-ORG       0.00      0.00      0.00         5
       I-ORG       0.85      0.86      0.86      2491
       B-LOC       0.00      0.00      0.00         6
       I-LOC       0.88      0.87      0.87      1916

   micro avg       0.97      0.97      0.97     46417
   macro avg       0.48      0.48      0.48     46417
weighted avg       0.97      0.97      0.97     46417



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
